In [12]:
import numpy as np
import torch
import random

In [13]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(2024)

In [14]:
ml100k_mf_full_rating = np.loadtxt('./ml100k_mf_full_rating.txt').astype(int)
ml100k_mf_full_rating

array([[2, 1, 2, ..., 1, 2, 1],
       [2, 2, 1, ..., 1, 3, 1],
       [3, 2, 1, ..., 1, 1, 1],
       ...,
       [3, 3, 1, ..., 1, 1, 1],
       [2, 2, 1, ..., 1, 1, 1],
       [2, 2, 3, ..., 1, 1, 3]])

In [15]:
np.bincount(ml100k_mf_full_rating.reshape(1, -1)[0])

array([     0, 834684, 383725, 228226,  99015,  40476])

In [47]:
p = 0.13
alpha = 0.5
ml100k_mf_full_rating = torch.tensor(ml100k_mf_full_rating)
ml100k_mf_full_rating


/tmp/ipykernel_55823/2746835213.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ml100k_mf_full_rating = torch.tensor(ml100k_mf_full_rating)


tensor([[2, 1, 2,  ..., 1, 2, 1],
        [2, 2, 1,  ..., 1, 3, 1],
        [3, 2, 1,  ..., 1, 1, 1],
        ...,
        [3, 3, 1,  ..., 1, 1, 1],
        [2, 2, 1,  ..., 1, 1, 1],
        [2, 2, 3,  ..., 1, 1, 3]])

In [48]:
beta = torch.clamp(6 - ml100k_mf_full_rating, max=4)

In [49]:
beta

tensor([[4, 4, 4,  ..., 4, 4, 4],
        [4, 4, 4,  ..., 4, 3, 4],
        [3, 4, 4,  ..., 4, 4, 4],
        ...,
        [3, 3, 4,  ..., 4, 4, 4],
        [4, 4, 4,  ..., 4, 4, 4],
        [4, 4, 3,  ..., 4, 4, 3]])

In [50]:
prob = p * (alpha ** beta)
prob

tensor([[0.0081, 0.0081, 0.0081,  ..., 0.0081, 0.0081, 0.0081],
        [0.0081, 0.0081, 0.0081,  ..., 0.0081, 0.0162, 0.0081],
        [0.0162, 0.0081, 0.0081,  ..., 0.0081, 0.0081, 0.0081],
        ...,
        [0.0162, 0.0162, 0.0081,  ..., 0.0081, 0.0081, 0.0081],
        [0.0081, 0.0081, 0.0081,  ..., 0.0081, 0.0081, 0.0081],
        [0.0081, 0.0081, 0.0162,  ..., 0.0081, 0.0081, 0.0162]])

In [51]:
prob.sum()

tensor(19457.1738)

In [52]:
ml100k_mf_full_rating.shape[0], ml100k_mf_full_rating.shape[1]

(943, 1682)

In [53]:
943 * 1682 * 0.01

15861.26

In [54]:
idx01 = torch.bernoulli(prob)
idx01

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [55]:
index1 = torch.nonzero(idx01 == 1)
index1

tensor([[   0,   16],
        [   0,   50],
        [   0,  348],
        ...,
        [ 942, 1528],
        [ 942, 1546],
        [ 942, 1623]])

In [56]:
len(np.unique(index1[:, 0].numpy()))

943

In [57]:
len(np.unique(index1[:, 1].numpy()))

1682

In [58]:
np.savetxt('./ground_truth.txt', ml100k_mf_full_rating.reshape(1, -1)[0].numpy(), fmt='%d')
np.savetxt('./propensity.txt', prob.reshape(1, -1)[0].numpy())
np.savetxt('./user_item.txt', index1.numpy(), fmt='%d')
np.savetxt('./y.txt', ml100k_mf_full_rating[index1[:, 0], index1[:, 1]].numpy(), fmt='%d')

In [59]:
ml100k_mf_full_rating[index1[:, 0], index1[:, 1]]

tensor([1, 2, 1,  ..., 1, 4, 1])